In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline


In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    #import math
    # Defining Empty arrays
    
    x_l = np.empty([0,0])
    y_l = np.empty([0,0])
    x_r = np.empty([0,0])
    y_r = np.empty([0,0])

    for line in lines:
    
        for x1,y1,x2,y2 in line:
        
            m = (y2 - y1)/(x2 - x1)
            # Length_line = (math.sqrt((x1 - x2)**2 + (y1 - y2)**2))
            # To seperate Lines as left and right
            # Might have to think about the length factor here
    
            if m < 0: 
        
                x_l = np.append(x_l, [x1, x2])
                y_l = np.append(y_l, [y1, y2])
        
            else:
        
                x_r = np.append(x_r, [x1, x2])
                y_r = np.append(y_r, [y1, y2])


    # Reference Lines for Lane End Positions

    y_lower  = 540
    y_higher = 340   # Changed from 320, for better performance

    ######################### Finding the best fit line for the Right Lane Line ############### 

    # Calculating the slope and intercept of the best fit line using least squares approach
    # Following the example from SciPy.org on numpy.linalg.lstsq
    
    A = np.vstack([x_r, np.ones(len(x_r))]).T
    m_fit_r,intercept_r = np.linalg.lstsq(A,y_r)[0]

    #plt.plot(x_r, y_r, 'o', label='Original data', markersize=10)
    #plt.plot(x_r, m_fit_r*x_r + intercept_r, 'r', label='Fitted Line')
    #plt.figure()
    #plt.show()

    # Now to Calculate the best fit Right lane Line, we use the reference y_lower = 540 (Image Size)
    # Calcualte x_lower 
    # Using the slope value of the fit to decide the correct lane line.
    # Rough Estimate of Right Lane Line slope was 0.57 for points (920,540) and (540,320) 
    
    if ((m_fit_r > 0.4) and (m_fit_r < 0.9)):
        
        x_lower_r = int((y_lower - intercept_r)/m_fit_r)
        
    else:
        
        x_lower_r = 875
    
    #print(x_lower_r)

    # Now to Calculate the best fit Right lane Line, we use the reference  y_higher = 340
    # Calcualte x_higher
    # Using the slope value of the fit to decide the correct lane line.
    # Rough Estimate of Right Lane Line slope was 0.57 for points (920,540) and (540,320)

    if ((m_fit_r > 0.4) and (m_fit_r < 0.9)):
        
        x_higher_r = int((y_higher - intercept_r)/m_fit_r)
        
    else:   
        
        x_higher_r = 560
        
    #x_higher_r = (y_higher - intercept_r)/m_fit_r
    
    #print(x_higher_r)

    #################### Finding the best fit line for the Left Lane Line #########################

    # Calculating the slope and intercept of the best fit line using least squares approach
    # Following the example from SciPy.org on numpy.linalg.lstsq
    
    A1 = np.vstack([x_l, np.ones(len(x_l))]).T
    m_fit_l,intercept_l = np.linalg.lstsq(A1,y_l)[0]

    #plt.plot(x_l, y_l, 'o', label='Original data', markersize=10)
    #plt.plot(x_l, m_fit_l*x_l + intercept_l, 'r', label='Fitted Line')
    #plt.figure()
    #plt.show()

    # Now to Calculate the best fit Left lane Line, we use the reference y_lower = 540 (Image Size)

    # Calcualte x_lower 
    # Using the slope value of the fit to decide the correct lane line.
    # Rough Estimate of Left Lane Line slope was -0.97 for points (190,540) and (420,320)   
    
    if ((m_fit_l < -0.5) and (m_fit_l >  -1)):
        
        x_lower_l = int((y_lower - intercept_l)/m_fit_l)
        
    else:
        
        x_lower_l = 190

    # Now to Calculate the best fit Left lane Line, we use the reference  y_higher = 340
    # Calcualte x_higher
    # Using the slope value of the fit to decide the correct lane line.
    # Rough Estimate of Left Lane Line slope was -0.97 for points (190,540) and (420,320)

    if ((m_fit_l < -0.5) and (m_fit_l > -1 )):
        
        x_higher_l = int((y_higher - intercept_l)/m_fit_l)
        
    else:
        
        x_higher_l = 420
        
 
    # Lets Draw the Right Lane Line 
    
    cv2.line(img, (x_lower_l, y_lower), (x_higher_l, y_higher), color, thickness)
    
    # Lets Draw the Right Lane Line 
    
    cv2.line(img, (x_lower_r, y_lower), (x_higher_r, y_higher), color, thickness)
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



In [3]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes
    
    gray = grayscale(image) #was color_select
    
    kernal_size = 7 # Must be an odd number 3, 5, 7...
    blur_gray = gaussian_blur(gray,kernal_size)
    
    low_threshold = 50
    high_threshold = 100
    edges = canny(blur_gray,low_threshold,high_threshold)
    
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(420,340),(580,340),(950,imshape[0])]],dtype=np.int32)
    test_mask_image = region_of_interest(edges, vertices)
    
    rho   = 1                    #distance resolution in meters 
    theta = 1*np.pi/180          #angular resolution in radians of the Hough grid 
    threshold = 30               #minimum number of votes (intersections in Hough Grid)  was 
    min_line_len = 100             #minimum number of pixels making up a line
    max_line_gap = 160             #maximum gap in pixels between connectable line segments
    final_image = hough_lines(test_mask_image, rho, theta,threshold, min_line_len, max_line_gap)
    
    result = weighted_img(final_image, image)
    
    return result

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [5]:
#Need to tweak my region of interest to be able to account for the cars lid that appears in the image frames.
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|████████████████████████████████████████| 251/251 [00:31<00:00,  7.70it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 35.9 s
